# Example of TFLearn to implement a Convolutional Neural Network

Have you ever developed a program that is able to find an object in an image? This was science fiction not so long ago, but machine learning is evolving so fast that now any developer can do it. Here is what you will need:
1. Training data: We will be using the public data from the [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html) which is automatically downloaded by the program.
1. A powerful machine (ideally with GPUs). But if not, it's ok, it will just be quite a bit slower ;)
1. A Neural Nets Library: We will be using Google's TensorFlow and it's high level abstraction called TFLearn
1. The algorithms and network topology known to produce good results. In this case we will be using a Convolutional Neural Network (or CNN for short). As you learn more and more about machine learning you will slowly be able to do this on your own. For now, trust what is shown below and play with it, tweeking the parameters and observing if the resulting model is more or less accurate.

This notebook has been created from the original source code at: https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_cifar10.py

An excellent article that provides an intuitive explanation of how CNN's work is https://t.co/Q7EcxtJcU

Go ahead and execute the program cell by cell (remember that the shortcut to execute a single cell is Shift+Enter).

In [1]:
# -*- coding: utf-8 -*-
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [2]:
# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

100.0% 170500096 / 170498071


('Succesfully downloaded', 'cifar-10-python.tar.gz', 170498071, 'bytes.')
File Extracted in Current Directory


In [3]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [4]:
# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [5]:
# Convolutional network building
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

In [6]:
network = conv_2d(network, 32, 3, activation='relu')

In [7]:
network = max_pool_2d(network, 2)

In [8]:
network = conv_2d(network, 64, 3, activation='relu')

In [9]:
network = conv_2d(network, 64, 3, activation='relu')

In [10]:
network = max_pool_2d(network, 2)

In [11]:
network = fully_connected(network, 512, activation='relu')

In [12]:
network = dropout(network, 0.5)

In [13]:
network = fully_connected(network, 10, activation='softmax')

In [14]:
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

In [15]:
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)

Our network is now ready and it time to train it with our sample data (our our validation set).

The code below does 10 iterations (n_epoch=10) which will be reasonably fast (a few mins) and will allow you to see how it works. However it will not produce a very accurate model. Increase the number of iterations to 40 for better results but beware that it will take longer and might even not work inside this docker image.

In [16]:
model.fit(X, Y, n_epoch=10, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

Training Step: 5210  | total loss: 0.68284
| Adam | epoch: 010 | loss: 0.68284 - acc: 0.7679 | val_loss: 0.64109 - val_acc: 0.7824 -- iter: 50000/50000
Training Step: 5210  | total loss: 0.68284
| Adam | epoch: 010 | loss: 0.68284 - acc: 0.7679 | val_loss: 0.64109 - val_acc: 0.7824 -- iter: 50000/50000
--


In [17]:
model.save("cifar-10.tfl")